In [ ]:
import time
from ast import parse

# define the path
GRAMMAR_PATH = "./atis/atis-grammar-cnf.cfg"
SENTENCES_PATH = "./atis/atis-test-sentences.txt"

In [ ]:
import nltk

grammar = nltk.data.load(GRAMMAR_PATH)  # load the grammar
raw_sentences = nltk.data.load(SENTENCES_PATH)  # load raw sentences
test_sentences = nltk.parse.util.extract_test_sentences(raw_sentences)  # extract test sentences

print(grammar)
print(raw_sentences)
for s in test_sentences:
    print(s)

In [ ]:
## test the parser
# initialize the parser
parser = nltk.parse.BottomUpChartParser(grammar)

#try to parse several test sentences
for sentence in test_sentences[:10]:
    try:
        trees = list(parser.parse(sentence[0]))
        if trees:
            print("Sentence: ", sentence[0], len(trees))
        else:
            print("Parsing failed for sentence: ", sentence[0])
    except Exception as e:
        print("Exception: ", e)
        print("Parsing failed for sentence: ", sentence[0])

In [ ]:
def CKY_reconize(sentence, grammar: nltk.grammar.CFG):
    # initialize the chart
    n = len(sentence)
    chart = [[set() for _ in range(n)] for _ in range(n)]

    # fill in the diagonal of the chart
    for i in range(n):
        # get the production whose rhs is the word i
        for production in grammar.productions(rhs=sentence[i]):
            # add the lhs of the production to the cell
            chart[i][i].add(production.lhs())

    # main body of the CKY algorithm
    # traverse the chart for each width b
    for b in range(2, n + 1):
        # for each start position i
        for i in range(0, n - b + 1):
            # for each left width k
            for k in range(0, b - 1):
                # for each non-terminal B and C
                # if there is a production B -> C in the grammar
                # add [B, C] to the cell
                for B in chart[i][i + k]:
                    for C in chart[i + k + 1][i + b - 1]:
                        # nltk grammar productions only accept one item in the rhs
                        for production in grammar.productions(rhs=B):
                            if production.rhs() == (B, C):
                                chart[i][i + b - 1].add(production.lhs())

    # if the start symbol is in chart[0][n-1], the sentence can be parsed
    if grammar.start() in chart[0][n - 1]:
        return True

    return False

In [ ]:
# Extra: Optimized running efficiency
def CKY_reconize_optimized(sentence, grammar: nltk.grammar.CFG):
    # initialize the chart
    n = len(sentence)
    chart = [[set() for _ in range(n)] for _ in range(n)]

    # Use production cache: map RHS -> LHS
    # Binary rules (e.g., A -> B C)
    binary_rules = dict()

    # add all binary rules to the cache
    for production in grammar.productions():
        if len(production.rhs()) == 2:
            if production.rhs() not in binary_rules:
                binary_rules[production.rhs()] = []
                binary_rules[production.rhs()].append(production.lhs())
            else:
                binary_rules[production.rhs()].append(production.lhs())

    # fill in the diagonal of the chart
    for i in range(n):
        # get the production whose rhs is the word i
        for production in grammar.productions(rhs=sentence[i]):
            # add the lhs of the production to the cell
            chart[i][i].add(production.lhs())

    # main body of the CKY algorithm
    # traverse the chart for each width b
    for b in range(2, n + 1):
        # for each start position i
        for i in range(0, n - b + 1):
            # for each left width k
            for k in range(0, b - 1):
                # for each non-terminal B and C
                # if there is a production B -> C in the grammar
                # add [B, C] to the cell
                for B in chart[i][i + k]:
                    for C in chart[i + k + 1][i + b - 1]:
                        # look up the cache for the lhs
                        for lhs in binary_rules.get((B, C), []):
                            chart[i][i + b - 1].add(lhs)

    # if the start symbol is in chart[0][n-1], the sentence can be parsed
    if grammar.start() in chart[0][n - 1]:
        return True

    return False

In [ ]:
for i in range(len(test_sentences)):
    print(i, test_sentences[i][0], end='\t')
    print(CKY_reconize(test_sentences[i][0], grammar))

In [ ]:
for i in range(len(test_sentences)):
    print(i, test_sentences[i][0], end='\t')
    print(CKY_reconize_optimized(test_sentences[i][0], grammar))

In [ ]:
# CKY parser
def CKY_parse(sentence, grammar: nltk.grammar.CFG):
    n = len(sentence)
    chart = [[set() for _ in range(n)] for _ in range(n)]
    # use dicts as back pointers
    back_pointers = [[{} for _ in range(n)] for _ in range(n)]

    binary_rules = dict()

    for production in grammar.productions():
        if len(production.rhs()) == 2:
            if production.rhs() not in binary_rules:
                binary_rules[production.rhs()] = []
                binary_rules[production.rhs()].append(production.lhs())
            else:
                binary_rules[production.rhs()].append(production.lhs())

    for i in range(n):
        for production in grammar.productions(rhs=sentence[i]):
            chart[i][i].add(production.lhs())
            back_pointers[i][i][production.lhs()] = [sentence[i]]

    # main body of the CKY algorithm
    for b in range(2, n + 1):
        for i in range(0, n - b + 1):
            for k in range(0, b - 1):
                for B in chart[i][i + k]:
                    for C in chart[i + k + 1][i + b - 1]:
                        for lhs in binary_rules.get((B, C), []):
                            chart[i][i + b - 1].add(lhs)

                            # add back pointers
                            if lhs not in back_pointers[i][i + b - 1]:
                                back_pointers[i][i + b - 1][lhs] = []
                            back_pointers[i][i + b - 1][lhs].append((B, C, i + k))

    # return back pointers
    if grammar.start() in chart[0][n - 1]:
        return True, back_pointers

    return False, None

In [ ]:
from nltk.tree import ImmutableTree


# extract the trees from the back pointers
def extract_trees(back_pointers, i, j, start_symbol):
    if i == j:
        return {ImmutableTree(start_symbol, [back_pointers[i][j][start_symbol][0]])}

    trees = set()

    for B, C, k in back_pointers[i][j][start_symbol]:
        left_trees = extract_trees(back_pointers, i, k, B)
        right_trees = extract_trees(back_pointers, k + 1, j, C)

        # combine the left and right subtrees
        for left in left_trees:
            for right in right_trees:
                trees.add(ImmutableTree(start_symbol, [left, right]))

    return trees

In [ ]:
# test the CKY parser
for sentence in test_sentences[:2]:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    if success:
        trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())
        print("Sentence: ", sentence[0], len(trees))
        for tree in trees:
            tree = nltk.tree.Tree.fromstring(str(tree))
            tree.pretty_print()
            tree.draw()
    else:
        print("Parsing failed for sentence: ", sentence[0])

In [ ]:
# Output the parsing results and compare with standard results
for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    try:
        tree_num = len(list(parser.parse(sentence[0])))
    except Exception as e:
        tree_num = 0

    trees = []
    if success:
        trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())
        print(sentence[0], '\t', len(trees))
    else:
        print(sentence[0], '\t', 0)

    if len(trees) != tree_num:
        print("Error: ", sentence[0], len(trees), tree_num)
        exit()

print("All tests passed!")

In [ ]:
# Extra: Figure out how to compute the number of parse trees with backpointers
def count_trees(back_pointers, i, j, start_symbol):
    if i == j:
        return 1

    count = 0

    for B, C, k in back_pointers[i][j][start_symbol]:
        left_count = count_trees(back_pointers, i, k, B)
        right_count = count_trees(back_pointers, k + 1, j, C)

        # just multiply the left trees number and right trees number
        count += left_count * right_count

    return count

In [ ]:
# Extra: test count_trees
for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    try:
        tree_num = len(list(parser.parse(sentence[0])))
    except Exception as e:
        tree_num = 0

    tree_count = 0
    if success:
        tree_count = count_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())

    if tree_count != tree_num:
        print("Error: ", sentence[0], tree_count, tree_num)
        exit()

    print(sentence[0], '\t', tree_count)

print("All tests passed!")

In [ ]:
# Extra: compare the efficiency improvement
import time

t1 = time.time()

for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    if success:
        trees = extract_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())

t2 = time.time()

for sentence in test_sentences:
    success, back_pointers = CKY_parse(sentence[0], grammar)
    if success:
        tree_num = count_trees(back_pointers, 0, len(sentence[0]) - 1, grammar.start())

t3 = time.time()

print("Time cost improvement: ", ((t2 - t1) - (t3 - t2)) / (t2 - t1))